In [2]:
from xsmc import XSMC, Segmentation
import xsmc._sampler
from xsmc.sampler import XSMCSampler
import numpy as np
from scipy.stats import sem
from scipy.interpolate import PPoly
import pandas as pd

import msprime as msp
import stdpopsim

import os
import time
from collections import defaultdict

In [7]:
import matplotlib
%matplotlib inline
matplotlib.rcParams.update({
    'font.family': 'Times New Roman',
    'text.usetex': True,
})
import matplotlib.pyplot as plt
import seaborn as sns
import plotnine as pn
from plotnine import *

In [22]:
folder = '../../../../exact_decoding_paper/figures/benchmark/'

In [9]:
# other supporting functions
L = int(5e6)  # length of simulated chromosome
mutation_rate = 1.4e-8  # mutation rate/bp/gen
recombination_rate = 1.4e-8
Ne = 1e4
n = 50  # sample size

theta = mutation_rate*Ne
rho = mutation_rate*Ne

def simulate(seed = 1):
    return msp.simulate(
        sample_size=n,
        mutation_rate=mutation_rate,
        recombination_rate=recombination_rate,
        Ne=Ne,
        length=L,
        random_seed=seed,
    )

In [17]:
def get_truth(ts, focal, panel):
    truth = np.empty(int(ts.sequence_length))
    for tree in ts.trees():
        start = int(tree.interval[0])
        end = int(tree.interval[1])
        truth[start:end] = tree.get_tmrca(focal, panel)
    return truth

def get_dist(truth, y):
    L = truth.size
    err_A = np.abs(truth - y).sum()/L
    err_B = np.abs(np.log10(truth/y)).sum()/L
    return err_A, err_B

def one_run(ts, focal, panel, err_A, err_B, run_times, epss):
    truth = get_truth(ts, focal, panel[0])
    for eps in epss:
        start = time.time()
        xs = XSMC(ts, focal=focal, panel=panel, theta=theta, rho_over_theta=rho/theta)
        X = xsmc._sampler.get_mismatches(xs._lwtc, xs.focal, xs.panel, xs.w)
        deltas = np.ones_like(X[0])
        # Perform sampling
        assert deltas.shape[0] == X.shape[1]
        xs._sampler = XSMCSampler(
            X=X,
            deltas=deltas,
            theta=xs.w * xs.theta,
            rho=xs.w * xs.rho,
            robust=xs.robust,
            eps=eps,
        )
        paths = xs.sample(k=100, seed=1)
        end = time.time()
        x = np.arange(ts.sequence_length)
        y = np.median([path.rescale(xs.theta / (2 * mutation_rate)).to_pp()(x) for path in paths], axis=0)
        A, B = get_dist(truth, y)
        err_A[eps].append(A)
        err_B[eps].append(B)
        run_times[eps].append(end-start)
        
def convert_to_array(d):
    for k,v in d.items():
        d[k] = np.array(v)

def run_sim(num_sim, epss):
    err_A = defaultdict(list)
    err_B = defaultdict(list)
    run_times = defaultdict(list)
    for i in range(1, num_sim):
        ts = simulate(i)
        nodes = [*range(n)]
        sample_set = [[x] for x in range(n)]
        g = ts.genealogical_nearest_neighbours(nodes, sample_set)
        focal, panel = np.unravel_index(g.argmax(), g.shape)
        one_run(ts, focal, [panel], err_A, err_B, run_times, epss)
    for d in [err_A, err_B, run_times]:
        convert_to_array(d)
        
    return err_A, err_B, run_times

def get_mean_se(d):
    d_mean = np.array([np.mean(v) for k,v in d.items()])
    d_se = np.array([sem(v) for k,v in d.items()])
    return d_mean, d_se

def format_entry(mean, se, dec_places):
    entry = f'{mean:.{dec_places}f} ({se:.{dec_places}f})'
    return entry
                                      
mfunc = np.vectorize(format_entry)

def save_table(tab, epss, name):
    index_list = epss
    tab['Truncation Parameter'] = index_list
    tab = tab.set_index('Truncation Parameter').T
    tab.to_latex(folder + name)
    display(tab)

In [18]:
epss = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
err_A, err_B, run_times = run_sim(25, epss)

In [19]:
mean_A, se_A = get_mean_se(err_A)
entries_abs = mfunc(mean_A, se_A, 2)

mean_B, se_B = get_mean_se(err_B)
entries_log = mfunc(mean_B, se_B, 4)

mean_time, se_time = get_mean_se(run_times)
entries_time = mfunc(mean_time, se_time, 4)

In [20]:
tab = pd.DataFrame(columns=epss, data=np.array([entries_abs, entries_log, entries_time]))
index_list = ['Err_A','Err_B','Time']
tab['Eps'] = index_list
tab = tab.set_index('Eps').T
tab

Eps,Err_A,Err_B,Time
0.01,3219.74 (608.23),0.6706 (0.1296),0.0835 (0.0062)
0.001,2979.61 (567.86),0.5708 (0.1042),0.1242 (0.0074)
0.0001,2905.50 (557.24),0.4799 (0.0825),0.1525 (0.0085)
1e-05,2877.94 (553.02),0.4004 (0.0691),0.1998 (0.0076)
1e-06,2864.44 (551.67),0.3268 (0.0477),0.3320 (0.0123)


In [24]:
tab.style.to_latex(folder + "truncation.tex")

In [9]:
entries_abs = mfunc(mean[:,:2], se[:,:2], 2)
entries_log = mfunc(mean[:,2:], se[:,2:], 4)

(array([0.05699095, 0.08025213, 0.1114525 , 0.17267444, 0.35247911]),
 array([0.00524902, 0.00667209, 0.00764058, 0.01098236, 0.03092825]))

In [26]:
mean = np.average(err_m, axis=2)
se = sem(err_mats, axis=2)

defaultdict(list,
            {0.01: array([[4.59289361e+00, 1.02144594e-01],
                    [4.80729746e+03, 2.82436044e-01],
                    [5.90757707e+03, 2.62333875e-01],
                    [3.20483765e+00, 3.81916457e-01],
                    [4.56073752e+03, 1.15798582e+00],
                    [1.81535749e+03, 2.38386939e-01],
                    [6.87530709e+03, 8.27644419e-01],
                    [2.09135837e+00, 1.09555272e+00],
                    [5.56857009e+03, 1.06088346e+00],
                    [7.19686416e+03, 1.47636142e+00],
                    [4.43742641e+03, 1.42470161e+00],
                    [6.05600138e-01, 1.02548372e-01],
                    [2.79185832e+03, 3.51056832e-01],
                    [6.14274754e+03, 1.14823004e+00],
                    [1.68940257e+03, 3.93653040e-01],
                    [1.43380035e+00, 4.80932011e-02],
                    [1.69169646e+03, 1.32758696e-01],
                    [2.08624284e+03, 4.01093076e-01],
    

In [41]:
i = 0
epss = [1e-2, 1e-3, 1e-4, 1e-5, 1e-7]
xss = {}
paths = {}
for eps in epss:
    xs = XSMC(tss[i], focal=indss[i][0], panel=[indss[i][1]], theta=theta, rho_over_theta=rho/theta)
    X = xsmc._sampler.get_mismatches(xs.ts, xs.focal, xs.panel, xs.w)
    deltas = np.ones_like(X[0])
    # Perform sampling
    assert deltas.shape[0] == X.shape[1]
    xs._sampler = XSMCSampler(
        X=X,
        deltas=deltas,
        theta=xs.w * xs.theta,
        rho=xs.w * xs.rho,
        robust=xs.robust,
        eps=eps,
    )
    xss[eps] = xs
    paths[eps] = xss[eps].sample(k=100, seed=1)

TypeError: object of type 'numpy.int64' has no len()

In [35]:
x = np.arange(xs.ts.sequence_length)
path_medians = {}
for eps in epss:
    path_medians[eps] = np.median([path.rescale(xs.theta / (2 * mutation_rate)).to_pp()(x) for path in paths[eps]], axis=0)

In [42]:
get_truth(tss[i], indss[i][0], indss[i][1])

TypeError: an integer is required (got type list)

In [39]:
[tree.get_tmrca(indss[i][0], indss[i][1]) for tree in tss[i].trees()]

[21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
 21.91427299504983,
